# The Flight Club

## Introduction

Welcome to the Flight Club! This project is designed to help track flight prices and receive notifications when there's a significant drop in fares. The system uses various APIs to search for flight deals, compare prices, and send alerts via email or WhatsApp to a list of registered users.
One caveat: Unfortunately, amadeus seems to provide faulty answers as of today, so that part of the project (providing real-live information transfer on budget deals) remains untested.

### How It Works

1. **Data Management:** The system retrieves and updates flight destination data and customer email addresses from a Google Sheet using the Sheety API.
2. **Flight Search:** It queries the Amadeus API to find the cheapest flights from a specified origin to various destinations.
3. **Notification:** If a significant price drop is detected, notifications are sent out via email or WhatsApp to all subscribed users.

## User Instructions

To run this notebook, follow these steps:

1. **Configure Environment Variables:**
   - Create a `.env` file in the same directory as this notebook
   - Add the required environment variables
   - Below is an example template for the `.env` file
2. **Enter Origin IATA of choice in main**
3. **Enter currency of choice in main**

In [ ]:
# plaintext
SHEETY_USERNAME=your_sheety_username
SHEETY_PASSWORD=your_sheety_password
SHEETY_PRICES_ENDPOINT=your_sheety_prices_endpoint
SHEETY_USERS_ENDPOINT=your_sheety_users_endpoint
AMADEUS_API_KEY=your_amadeus_api_key
AMADEUS_SECRET=your_amadeus_secret
EMAIL_PROVIDER_SMTP_ADDRESS=your_email_provider_smtp_address
MY_EMAIL=your_email
MY_EMAIL_PASSWORD=your_email_password
TWILIO_SID=your_twilio_sid
TWILIO_AUTH_TOKEN=your_twilio_auth_token
TWILIO_VIRTUAL_NUMBER=your_twilio_virtual_number
TWILIO_VERIFIED_NUMBER=your_twilio_verified_number

## Imported Libraries

In [ ]:
import datetime as dt
import os
import requests
import smtplib

- datetime as dt: Imports the datetime module and aliases it as dt for convenient access
- os: Provides a way to interact with the operating system, including reading environment variables
- requests: Used for making HTTP requests to APIs
- smtplib: Supports sending emails using the Simple Mail Transfer Protocol (SMTP)

In [ ]:
from datetime import datetime
from dotenv import load_dotenv
from pprint import pprint
from requests.auth import HTTPBasicAuth
from twilio.rest import Client

- datetime import datetime: Specifically imports the datetime class from the datetime module
- dotenv import load_dotenv: Loads environment variables from a .env file
- pprint import pprint: Pretty prints data structures for easier readability
- requests.auth import HTTPBasicAuth: Handles HTTP Basic Authentication for API requests
- twilio.rest import Client: Provides access to Twilio's API for sending SMS and WhatsApp messages

## DataManager Class

In [ ]:
# Load environment variables from .env file
load_dotenv()

class DataManager:
    def __init__(self):
        # Initialize credentials and endpoints from environment variables
        self._user = os.environ["SHEETY_USERNAME"]
        self._password = os.environ["SHEETY_PASSWORD"]
        self.prices_endpoint = os.environ["SHEETY_PRICES_ENDPOINT"]
        self.users_endpoint = os.environ["SHEETY_USERS_ENDPOINT"]
        self._authorization = HTTPBasicAuth(self._user, self._password)
        self.destination_data = {}
        self.customer_data = {}

    def get_destination_data(self):
        # Use the Sheety API to GET all the data in the sheet
        response = requests.get(url=self.prices_endpoint, auth=self._authorization)
        data = response.json()
        self.destination_data = data["prices"]
        return self.destination_data

    def update_destination_codes(self):
        # Update the Google Sheet with the IATA codes
        for city in self.destination_data:
            new_data = {
                "price": {
                    "iataCode": city["iataCode"]
                }
            }
            response = requests.put(
                url=f"{self.prices_endpoint}/{city['id']}",
                json=new_data
            )
            print(response.text)

    def get_customer_emails(self):
        # Retrieve customer emails from the Sheety API
        response = requests.get(url=self.users_endpoint, auth=self._authorization)
        data = response.json()
        self.customer_data = data["users"]
        return self.customer_data

- __init__(self): Initializes class variables with environment variables for authentication and endpoints
- get_destination_data(self): Fetches destination data from Sheety. This includes retrieving and storing destination details from a Google Sheet
- update_destination_codes(self): Updates the IATA codes in the Sheety data based on the destination information. Uses the PUT method to modify existing entries in the Google Sheet
- get_customer_emails(self): Retrieves customer email addresses from Sheety. This list is used to send notifications when a flight deal is found

## FlightData Class and find_cheapest_flight Function

In [ ]:
class FlightData:
    def __init__(self, price, origin_airport, destination_airport, out_date, return_date, stops):
        self.price = price
        self.origin_airport = origin_airport
        self.destination_airport = destination_airport
        self.out_date = out_date
        self.return_date = return_date
        self.stops = stops

def find_cheapest_flight(data):
    if data is None or not data['data']:
        print("No flight data")
        return FlightData(
            price="N/A",
            origin_airport="N/A",
            destination_airport="N/A",
            out_date="N/A",
            return_date="N/A",
            stops="N/A"
        )

    first_flight = data['data'][0]
    lowest_price = float(first_flight["price"]["grandTotal"])
    nr_stops = len(first_flight["itineraries"][0]["segments"]) - 1
    origin = first_flight["itineraries"][0]["segments"][0]["departure"]["iataCode"]
    destination = first_flight["itineraries"][0]["segments"][nr_stops]["arrival"]["iataCode"]
    out_date = first_flight["itineraries"][0]["segments"][0]["departure"]["at"].split("T")[0]
    return_date = first_flight["itineraries"][1]["segments"][0]["departure"]["at"].split("T")[0]

    cheapest_flight = FlightData(lowest_price, origin, destination, out_date, return_date, nr_stops)

    for flight in data["data"]:
        price = float(flight["price"]["grandTotal"])
        if price < lowest_price:
            lowest_price = price
            origin = flight["itineraries"][0]["segments"][0]["departure"]["iataCode"]
            destination = flight["itineraries"][0]["segments"][nr_stops]["arrival"]["iataCode"]
            out_date = flight["itineraries"][0]["segments"][0]["departure"]["at"].split("T")[0]
            return_date = flight["itineraries"][1]["segments"][0]["departure"]["at"].split("T")[0]
            cheapest_flight = FlightData(lowest_price, origin, destination, out_date, return_date, nr_stops)
            print(f"Lowest price to {destination} is £{lowest_price}")

    return cheapest_flight

- find_cheapest_flight Function: Analyzes the flight data to identify the cheapest flight option
    - Checks if flight data is empty or not available
    - Extracts the lowest price and flight details from the response
    - Compares prices of all available flights and updates the FlightData instance with the cheapest option

## FlightSearch Class

In [ ]:
# Load environment variables from .env file
load_dotenv()

IATA_ENDPOINT = "https://test.api.amadeus.com/v1/reference-data/locations/cities"
FLIGHT_ENDPOINT = "https://test.api.amadeus.com/v2/shopping/flight-offers"
TOKEN_ENDPOINT = "https://test.api.amadeus.com/v1/security/oauth2/token"

class FlightSearch:
    def __init__(self):
        self._api_key = os.environ["AMADEUS_API_KEY"]
        self._api_secret = os.environ["AMADEUS_SECRET"]
        self._token = self._get_new_token()

    def _get_new_token(self):
        header = {
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        body = {
            'grant_type': 'client_credentials',
            'client_id': self._api_key,
            'client_secret': self._api_secret
        }
        response = requests.post(url=TOKEN_ENDPOINT, headers=header, data=body)
        return response.json()['access_token']

    def get_destination_code(self, city_name):
        headers = {"Authorization": f"Bearer {self._token}"}
        query = {
            "keyword": city_name,
            "max": "2",
            "include": "AIRPORTS",
        }
        response = requests.get(
            url=IATA_ENDPOINT,
            headers=headers,
            params=query
        )
        try:
            code = response.json()["data"][0]['iataCode']
        except (IndexError, KeyError):
            return "N/A"
        return code

    def check_flights(self, origin_city_code, destination_city_code, from_time, to_time, is_direct=True):
        headers = {"Authorization": f"Bearer {self._token}"}
        query = {
            "originLocationCode": origin_city_code,
            "destinationLocationCode": destination_city_code,
            "departureDate": from_time.strftime("%Y-%m-%d"),
            "returnDate": to_time.strftime("%Y-%m-%d"),
            "adults": 1,
            "nonStop": "true" if is_direct else "false",
            "currencyCode": "GBP",
            "max": "10",
        }
        response = requests.get(
            url=FLIGHT_ENDPOINT,
            headers=headers,
            params=query,
        )
        if response.status_code != 200:
            return None
        return response.json()

- __init__(self): Initializes with API credentials and fetches a new token for authorization
- _get_new_token(self): Requests a new access token from the Amadeus API. Tokens are required for authenticated requests
- get_destination_code(self, city_name): Retrieves the IATA code for a given city name by querying the Amadeus API
- check_flights(self, origin_city_code, destination_city_code, from_time, to_time, is_direct=True): Searches for flights between the origin and destination, filtering by direct flights or not

## NotificationManager Class

In [ ]:
class NotificationManager:
    def __init__(self):
        self.smtp_address = os.environ["EMAIL_PROVIDER_SMTP_ADDRESS"]
        self.email = os.environ["MY_EMAIL"]
        self.email_password = os.environ["MY_EMAIL_PASSWORD"]
        self.twilio_virtual_number = os.environ["TWILIO_VIRTUAL_NUMBER"]
        self.twilio_verified_number = os.environ["TWILIO_VERIFIED_NUMBER"]
        self.client = Client(os.environ['TWILIO_SID'], os.environ["TWILIO_AUTH_TOKEN"])
        self.connection = smtplib.SMTP(self.smtp_address)

    def send_sms(self, message_body):
        message = self.client.messages.create(
            from_=self.twilio_virtual_number,
            body=message_body,
            to=self.twilio_verified_number
        )
        print(message.sid)

    def send_whatsapp(self, message_body):
        message = self.client.messages.create(
            from_=f'whatsapp:{self.twilio_virtual_number}',
            body=message_body,
            to=f'whatsapp:{self.twilio_verified_number}'
        )
        print(message.sid)

    def send_emails(self, email_list, email_body):
        with self.connection:
            self.connection.starttls()
            self.connection.login(self.email, self.email_password)
            for email in email_list:
                self.connection.sendmail(
                    from_addr=self.email,
                    to_addrs=email,
                    msg=f"Subject:New Low Price Flight!\n\n{email_body}".encode('utf-8')
                )

- __init__(self): Initializes the class with email and Twilio credentials. Sets up connections for sending emails and SMS
- send_sms(self, message_body): Sends an SMS using Twilio's API
- send_whatsapp(self, message_body): Sends a WhatsApp message using Twilio's API (if WhatsApp is preferred)
- send_emails(self, email_list, email_body): Sends emails to a list of recipients with the specified message body

## Main Program

In [ ]:
# ==================== Set up the Flight Search ====================

data_manager = DataManager()
sheet_data = data_manager.get_destination_data()
flight_search = FlightSearch()
notification_manager = NotificationManager()

# Set your origin airport
ORIGIN_CITY_IATA = "FRA" # Example IATA Code

# ==================== Update the Airport Codes in Google Sheet ====================

for row in sheet_data:
    if row["iataCode"] == "":
        row["iataCode"] = flight_search.get_destination_code(row["city"])
        # slowing down requests to avoid rate limit
        time.sleep(2)
print(f"sheet_data:\n {sheet_data}")

data_manager.destination_data = sheet_data
data_manager.update_destination_codes()

# ==================== Retrieve your customer emails ====================

customer_data = data_manager.get_customer_emails()
customer_email_list = [row["whatIsYourEmail?"] for row in customer_data]

# ==================== Search for direct flights  ====================

tomorrow = datetime.now() + timedelta(days=1)
six_month_from_today = datetime.now() + timedelta(days=(6 * 30))

for destination in sheet_data:
    print(f"Getting direct flights for {destination['city']}...")
    flights = flight_search.check_flights(
        ORIGIN_CITY_IATA,
        destination["iataCode"],
        from_time=tomorrow,
        to_time=six_month_from_today
    )
    cheapest_flight = find_cheapest_flight(flights)
    print(f"{destination['city']}: €{cheapest_flight.price}")
    # Slowing down requests to avoid rate limit
    time.sleep(2)

    # ==================== Search for indirect flight if N/A ====================

    if cheapest_flight.price == "N/A":
        print(f"No direct flight to {destination['city']}. Looking for indirect flights...")
        stopover_flights = flight_search.check_flights(
            ORIGIN_CITY_IATA,
            destination["iataCode"],
            from_time=tomorrow,
            to_time=six_month_from_today,
            is_direct=False
        )
        cheapest_flight = find_cheapest_flight(stopover_flights)
        print(f"Cheapest indirect flight price is: €{cheapest_flight.price}")

    # ==================== Send Notifications and Emails  ====================

    if cheapest_flight.price != "N/A" and cheapest_flight.price < destination["lowestPrice"]:
        if cheapest_flight.stops == 0:
            message = f"Low price alert! Only € {cheapest_flight.price} to fly direct "\
                      f"from {cheapest_flight.origin_airport} to {cheapest_flight.destination_airport}, "\
                      f"on {cheapest_flight.out_date} until {cheapest_flight.return_date}."
        else:
            message = f"Low price alert! Only € {cheapest_flight.price} to fly "\
                      f"from {cheapest_flight.origin_airport} to {cheapest_flight.destination_airport}, "\
                      f"with {cheapest_flight.stops} stop(s) "\
                      f"departing on {cheapest_flight.out_date} and returning on {cheapest_flight.return_date}."

        print(f"Check your email. Lower price flight found to {destination['city']}!")

        notification_manager.send_whatsapp(message_body=message)
        notification_manager.send_emails(email_list=customer_email_list, email_body=message)

- Set up the Flight Search: Initializes the DataManager, FlightSearch, and NotificationManager instances
- Update the Airport Codes: Updates IATA codes in the destination data and refreshes the Google Sheet with these codes
- Retrieve Customer Emails: Gets the list of customer emails from Sheety to send notifications
- Search for Flights: Searches for direct flights, and if none are found, searches for indirect flights. Compares prices to find the cheapest option
- Send Notifications and Emails: Sends notifications via WhatsApp and emails if a cheaper flight is found than previously recorded

## Conclusion

Embarking on this project was an enriching journey that tested and expanded my skills in multiple areas of software development. Throughout the process, I encountered several challenges and learned a great deal about integrating various technologies.

One significant hurdle was dealing with internal errors in the responses from the Amadeus API. Despite meticulous troubleshooting and exploring available documentation, I discovered that some issues were beyond my immediate control due to limitations in the API itself. This experience underscored the importance of flexibility and adaptability when working with external services, where perfect control is not always achievable.

Integrating Google Sheets via the Sheety API posed another interesting challenge. It required me to interface with a non-standard API to manage and update data seamlessly. The process of mapping Google Sheets data to application logic, ensuring proper authentication, and handling updates was both complex and rewarding. This integration allowed me to efficiently manage destination data and streamline the entire workflow, demonstrating the power of cloud-based tools in modern development.

Working with Google Forms was an enjoyable aspect of the project, as it provided a user-friendly way to collect and manage customer emails. It was gratifying to see how easily data could be collected and utilized to enhance the functionality of the application. The seamless integration of user input through forms added a layer of interactivity and responsiveness to the project.

Overall, this project was a significant mental workout that pushed me to my limits and beyond. I relished the opportunity to flex my problem-solving skills, dive into complex API interactions, and integrate various systems. The satisfaction of overcoming obstacles and achieving a functional, integrated solution was immensely fulfilling. The challenges I faced not only tested my technical abilities but also deepened my understanding of modern development practices, leaving me with quite a sense of accomplishment and growth.